In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-123911")
exp = Experiment(workspace=ws, name="exphyp_aut1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-123911
Azure region: southcentralus
Subscription id: de8aba62-c352-42be-b980-2faedf08ead8
Resource group: aml-quickstarts-123911


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

cluster_name = "AIcompinst3"



try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())
    

Found existing compute target
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-10-31T15:21:24.629000+00:00', 'errors': None, 'creationTime': '2020-10-31T15:21:20.964835+00:00', 'modifiedTime': '2020-10-31T15:21:37.145430+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [5]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = 'de8aba62-c352-42be-b980-2faedf08ead8'
resource_group = 'aml-quickstarts-123911'
workspace_name = 'quick-starts-ws-123911'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Bank_Data')

In [6]:
import os

if "training" not in os.listdir():
    os.mkdir("./training")

In [9]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os


# Specify parameter sampler
ps = RandomParameterSampling(
    {
        # There's only one parameter, so grid sampling will try each value - with multiple parameters it would try every combination
        '--C': uniform( 0.01,1.0),
        '--max_iter': choice(range(95,105))
    }
)


# Specify a Policy
policy = BanditPolicy(slack_amount = 0.2,
                      evaluation_interval=1,
                      delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=os.getcwd()+'/training',
              entry_script='train.py',
              inputs=[dataset.as_named_input('Bank_Data')], # Pass the dataset as an input...
              compute_target = compute_target)
              


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(estimator=est,
                                      hyperparameter_sampling=ps, 
                                      policy=policy, 
                                      primary_metric_name='Accuracy', 
                                      primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                                      max_total_runs=25,
                                      max_concurrent_runs=4)



In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyprun = exp.submit(hyperdrive_config)
RunDetails(hyprun).show()


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [12]:
import joblib
# Get your best run and save the model from that run.

model = hyprun.get_best_run_by_primary_metric()
model = model.register_model(model_name='hyptun_bank', model_path='outputs/hyptun_model.joblib')

In [13]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')


In [25]:
from train import clean_data

# Use the clean_data function to clean your data.


x = clean_data(ds)### YOUR DATA OBJECT HERE ###)



In [28]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=x,
    label_column_name='y',
    n_cross_validations=3)

In [29]:
# Submit your automl run


auto_run = exp.submit(config=automl_config)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


In [30]:
# Retrieve and save your best automl model.


best_run, fitted_model = auto_run.get_output()
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

best_run.register_model(model_path='outputs/auto_model.pkl', model_name='bank_auto',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})